In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import nltk
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix
from TwitterSearch import TwitterUserOrder, TwitterSearch
import json
import random
from scipy.sparse.coo import coo_matrix

nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

% matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adrianofreitas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adrianofreitas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Funções úteis

In [2]:
def get_new_from_url(url: str) -> str:
    """Retorna o título e conteúdo de uma matéria do G1"""
    text = requests.get(url).text
    soup = BeautifulSoup(text, 'lxml')

    headline = soup.find_all('h1', class_='content-head__title')[0].text
    article = soup.find_all('article')[0].text
    
    return headline, article

In [3]:
def nomalize(text: str) -> str:
    """Normaliza o texto de uma matéria aplicando os seguintes passos:
    1- Converte tudo para minúculo
    2- Remove tags, caracteres especiais e dígios
    3- Aplica as técnicas Stemming e Lemmatisation
    """
    stop_words = stopwords.words('portuguese')
    new_text = text.lower()

    # removendo tags
    new_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",new_text)

    # removendo caracteres especiais e dígitos
    new_text=re.sub("(\\d|\\W)+"," ",new_text)

    new_text = new_text.split()
    
    # Stemming
    ps=PorterStemmer()
    
    # Lemmatisation
    lem = WordNetLemmatizer()
    
    new_text = [lem.lemmatize(word) for word in new_text if not word in stop_words] 
    new_text = " ".join(new_text)

    return new_text

In [4]:
def sort(coo_matrix: coo_matrix) -> list:
    """Ordena as keywords calculadas em order decrescente"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [5]:
def extract_top_n(feature_names: dict, sorted_items: list , topn: int = 10):
    """Captura as top N features e seu score"""
    
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    
    for i, score in sorted_items:
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[i])
 
    results= {}
    for i in range(len(feature_vals)):
        results[feature_vals[i]]=score_vals[i]
    
    return results

Criando um corpus baseado nas últimas notícias twitadas

In [6]:
config = json.load(open('config.json'))

tuo = TwitterUserOrder('g1')
ts = TwitterSearch(
    consumer_key = config['credentials']['consumer-key'],
    consumer_secret = config['credentials']['consumer-secret'],
    access_token = config['credentials']['access-token'],
    access_token_secret = config['credentials']['access-token-secret']
)

corpus = []
news = []
total_news = 0
for tweet in ts.search_tweets_iterable(tuo):
    if not tweet['truncated']:
        try:
            for url in tweet['entities']['urls']:
                headline, text = get_new_from_url(url['url'])
                corpus.append(nomalize(text))
                news.append({
                    'url': url['url'],
                    'headline': headline,
                    'text': text
                })

            total_news += 1
            print('tweet aproveitado: {}'.format( tweet['text'] ) )
        except:
            print('tweet não aproveitado: {}'.format( tweet['text'] ) )
    
    if total_news > config['n-news']:
        break

tweet aproveitado: Roberto Silva doou R$ 8 mil para eleição de Valdecy da Saúde (PHS) em 2018, e agora foi nomeado 'adido' https://t.co/2dls6Hi73C #G1
tweet aproveitado: "Ele já fez isso outras vezes, as pessoas estão começando a denunciar. É um serial killer de filhotes felinos" https://t.co/78CTcKZTBZ #G1
tweet aproveitado: Polícia investiga suposta rede de proteção a João de Deus https://t.co/vJ5zEgN90p #G1
tweet aproveitado: Segundo o Conar, "numerosas denúncias de consumidores" motivaram a abertura do processo https://t.co/k4INkAER7l #G1
tweet aproveitado: Parlamento britânico aumenta seus próprios poderes e irá votar opções ao acordo de May sobre Brexit https://t.co/azpAgEulNv #G1
tweet aproveitado: Avião que deveria ir para a Alemanha pousa na Escócia por engano https://t.co/pYEMv4lsnq #G1
tweet aproveitado: Príncipe Charles passeia por Havana e inaugura estátua de Shakespeare https://t.co/0wyHXQRAdN #G1
tweet aproveitado: Lava Jato denuncia operador financeiro ligado ao PSDB e 

tweet aproveitado: As aulas estão programadas para começar nesta terça-feira (26) https://t.co/niblKyZXky #G1
tweet aproveitado: #Enem2019: pedidos de isenção na taxa de inscrição começam em 1º de abril https://t.co/HMDxWR8CZW #G1 https://t.co/1T4YGyz3C0
tweet aproveitado: A investigação deve se concentrar no que poderia ou deveria ter sido feito para prevenir o ataque https://t.co/dTGomk1VCq #G1
tweet aproveitado: Forças Armadas israelenses culpam o Hamas pelo disparo https://t.co/6Jra66iebD #G1
tweet aproveitado: Polícia investiga as circunstâncias em que o garoto foi ferido https://t.co/UXw93zBLww #G1
tweet aproveitado: Ele doa 80% de seu salário para apoiar os estudos dos seus alunos https://t.co/jPWDWZToB4 #G1
tweet aproveitado: Veja lista de 4 recomendações para não cair em ciladas na hora de escolher investimentos https://t.co/tu8kNa10q2 #G1
tweet aproveitado: RT @showdavida: Balão de homenagem a vítimas de tragédia em escola vai parar em Suzano. Foi uma viagem de mais de 17 km.

Criando um vocabulário com todas as palavras conhecidas

In [7]:
stop_words = stopwords.words('portuguese')
cv = CountVectorizer(stop_words=stop_words, max_features=10000, ngram_range=(1,3))
X = cv.fit_transform(corpus)
list(cv.vocabulary_.keys())[:10]

['pm',
 'cedeu',
 'policiais',
 'outros',
 'órgãos',
 'doi',
 'agosto',
 'subtenente',
 'roberto',
 'araújo']

Selecionando uma notícia aleatória para ser analisada

In [8]:
selected_new = random.randint(0, len(corpus)-1)
print(news[selected_new]['headline'])

Hamas anuncia cessar-fogo com Israel após escalada de confrontos nas últimas horas


Utilizando TF-IDF para identificar as palavras-chave do artigo selecionado

In [9]:
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X)

feature_names = cv.get_feature_names()

tf_idf_vector = tfidf_transformer.transform(cv.transform([corpus[selected_new]]))

Ranqueando as palavras e exibindo os resultados

In [10]:
sorted_items = sort(tf_idf_vector.tocoo())

keywords = extract_top_n(feature_names, sorted_items, config['n-keywords'])

print("Matéria: {} [{}]\n{}".format(news[selected_new]['headline'], news[selected_new]['url'], news[selected_new]['text']))
print("\nKeywords:")
for k in keywords:
    print(k,keywords[k])

Matéria: Hamas anuncia cessar-fogo com Israel após escalada de confrontos nas últimas horas [https://t.co/Ql0GQ4IdXk]
                   Chamas são vistas durante ataque de Israel à Gaza, na segunda-feira (25) — Foto: Reuters/Mohammed Ajour           A organização palestina Hamas anunciou nesta segunda-feira (25) que alcançou um cessar-fogo com Israel com mediação do Egito, após horas de intenso confronto, a apenas duas semanas das eleições israelenses.      "Os esforços egípcios tiveram sucesso", alcançando "um cessar-fogo entre a ocupação e as facções da resistência", disse o porta-voz da organização, Fawzi Barhum.     Segundo a France Presse, Israel ainda não confirmou a informação.     O confronto se intensificou nas últimas horas após um foguete lançado da Faixa de Gaza ter atingido esta manhã uma casa na comunidade de Mishmeret, na região central de Israel, deixando sete feridos, entre eles uma mulher de 60 anos, uma de 30 anos, uma menina de 12 anos e duas crianças, de 3 e 1 ano